In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import xlrd
import datetime
from datetime import timedelta

In [4]:
data = pd.read_excel('Data.xlsx')
#data.head()

In [ ]:
entries = "Total rows: {0}".format(len(data))
print (entries)

In [ ]:
plt.figure(figsize = (18,9))
plt.plot(range(data.shape[0]),(data['PV Production (kW)']))
plt.xticks(range(0,data.shape[0],500),data['Date'].loc[::500],rotation=45)
plt.xlabel('Date',fontsize=18)
plt.ylabel('PV Production',fontsize=18)
plt.show()

In [ ]:
x = data[['Date']]
y = data[['PV Production (kW)']].rolling(24).mean()

plt.figure(figsize=(20,10))
plt.xlabel('Date',fontsize=18)
plt.ylabel('PV Production',fontsize=18)
plt.plot(x,y,linewidth=3)
#plt.show()

In [ ]:
x = data[['Date']]
y = data[['Load']].diff(24)

plt.figure(figsize=(20,10))
plt.xlabel('Date',fontsize=18)
plt.ylabel('PV Production (kW)',fontsize=18)
plt.plot(x,y,linewidth=3)
#plt.show()

In [ ]:
from pandas.plotting import autocorrelation_plot
from matplotlib.pyplot import figure
plt.figure(linewidth=3, figsize=(20,15))
autocorrelation_plot(data[['PV Production (kW)']]).plot();

In [ ]:
from matplotlib import pyplot
series = data['PV Production (kW)']
data['PV Production (kW)'].hist(linewidth=2,figsize=(20,15))
plt.ylabel('Distribution',fontsize=18)
plt.xlabel('PV Production (kW)',fontsize=18)
pyplot.show()

In [ ]:
series.plot(kind='kde',linewidth=2,figsize=(20,15))
plt.ylabel('Density',fontsize=18)
plt.xlabel('PV Production (kW)',fontsize=18)
pyplot.show()

In [ ]:
from pandas.plotting import lag_plot
plt.figure(linewidth=2, figsize=(20,15))
lag_plot(series)
pyplot.show()

In [ ]:
import numpy
import matplotlib.pyplot as plt
import pandas
import math
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM, RNN, GRU, SimpleRNN
from keras.layers import Dropout  
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error

In [ ]:
data = data[['Date','PV Production (kW)']]
data.head()

In [ ]:
minVal = min(data['PV Production (kW)']);
maxVal = max(data['PV Production (kW)']);
v = data['PV Production (kW)'].tolist()
norm=[]
for x in v:
    x= (x- minVal) / ( maxVal - minVal )
    norm.append(x)
data['PV Production (kW)'] = norm
data.head()

In [ ]:
data.rename(columns={'PV Production (kW)':'PV'}, inplace=True)

In [ ]:
for obs in range(1,6):
    data["T_" + str(obs)] = data.PV.shift(obs)
data.fillna(0.00,inplace=True)

In [ ]:
data.head()

In [ ]:
training_data = data[data.Date < pd.to_datetime("10/01/2017")]

val_mask = (data.Date >= pd.to_datetime("10/01/2017")) & (data.Date < pd.to_datetime("11/01/2017"))
val_data = data.loc[val_mask]

test_data = data[data.Date >= pd.to_datetime("11/01/2017")]

training_data = training_data.drop(["Date"],axis=1)
val_data = val_data.drop(["Date"],axis=1)
test_data = test_data.drop(["Date"],axis=1)
training_data.tail(3)

In [ ]:
x_train = training_data.drop(["PV"],axis=1)
y_train = training_data.PV

x_val = val_data.drop(["PV"],axis=1)
y_val = val_data.PV

x_test = test_data.drop(["PV"],axis=1)
y_test = test_data.PV

In [ ]:
model_lstm = Sequential()
model_lstm.add(LSTM(4, return_sequences=False, input_shape=(1,5)))
model_lstm.add(Dropout(0.1))  

#model_k.add(LSTM(6, return_sequences=False))  
#model_k.add(Dropout(0.1))  

model_lstm.add(Dense(1))
model_lstm.compile(loss='mean_squared_error', optimizer='adam')

In [ ]:
x_train_reshaped = x_train.values.reshape((x_train.shape[0], 1, x_train.shape[1]))
x_val_reshaped = x_val.values.reshape((x_val.shape[0], 1, x_val.shape[1]))
x_test_reshaped = x_test.values.reshape((x_test.shape[0], 1, x_test.shape[1]))

In [ ]:
historyLSTM = model_lstm.fit(x_train_reshaped, y_train, validation_data=(x_val_reshaped, y_val), epochs=20, batch_size=64, verbose=2)

In [ ]:
plt.plot(historyLSTM.history['loss'], label='train', color = 'r')
plt.plot(historyLSTM.history['val_loss'], label='val')
plt.ylabel('Loss',fontsize=12)
plt.xlabel('Epoch',fontsize=12)
plt.legend()

In [ ]:
resLSTM = model_lstm.predict(x_test_reshaped)
test_data["DL_PRED"] = resLSTM
from numpy import sqrt
sqrt(mean_squared_error(test_data.PV,test_data.DL_PRED))

In [ ]:
model_g = Sequential()
model_g.add(GRU(4, return_sequences=False, input_shape=(1,5)))
model_g.add(Dropout(0.1))  

#model_k.add(LSTM(6, return_sequences=False))  
#model_k.add(Dropout(0.1))  

model_g.add(Dense(1))
model_g.compile(loss='mean_squared_error', optimizer='adam')

In [ ]:
history_gru = model_g.fit(x_train_reshaped, y_train, validation_data=(x_val_reshaped, y_val), epochs=20, batch_size=64, verbose=2)

In [ ]:
plt.plot(history_gru.history['loss'], label='train', color = 'r')
plt.plot(history_gru.history['val_loss'], label='val')
plt.ylabel('Loss',fontsize=12)
plt.xlabel('Epoch',fontsize=12)
plt.legend()

In [ ]:
resGRU = model_g.predict(x_test_reshaped)
test_data["DL_PRED_GRU"] = resGRU
from numpy import sqrt
sqrt(mean_squared_error(test_data.PV,test_data.DL_PRED_GRU))

In [ ]:
model_rnn = Sequential()
model_rnn.add(SimpleRNN(4, return_sequences=False, input_shape=(1,5)))
model_rnn.add(Dropout(0.1))  

#model_k.add(LSTM(6, return_sequences=False))  
#model_k.add(Dropout(0.1))  

model_rnn.add(Dense(1))
model_rnn.compile(loss='mean_squared_error', optimizer='adam')

In [ ]:
history_rnn = model_rnn.fit(x_train_reshaped, y_train, validation_data=(x_val_reshaped, y_val), epochs=20, batch_size=64, verbose=2)

In [ ]:
plt.plot(history_rnn.history['loss'], label='train', color = 'r')
plt.plot(history_rnn.history['val_loss'], label='val')
plt.ylabel('Loss',fontsize=12)
plt.xlabel('Epoch',fontsize=12)
plt.legend()

In [ ]:
res_rnn = model_rnn.predict(x_test_reshaped)
test_data["DL_PRED_RNN"] = res_rnn
sqrt(mean_squared_error(test_data.PV,test_data.DL_PRED_RNN))

In [ ]:
plt.plot(history_rnn.history['loss'], label='trainRNN', color = '#FF0000')
plt.plot(history_rnn.history['val_loss'], label='valRNN', color = '#FF0000')
plt.plot(historyLSTM.history['loss'], label='trainLSTM', color = '#09CF00')
plt.plot(historyLSTM.history['val_loss'], label='valLSTM', color = '#09CF00')
plt.plot(history_gru.history['loss'], label='trainGRU', color = '#0064FF')
plt.plot(history_gru.history['val_loss'], label='valGRU', color = '#0064FF')
plt.ylabel('Loss',fontsize=12)
plt.xlabel('Epoch',fontsize=12)
plt.legend()

In [ ]:
plt.figure(figsize=(20,10))
#test_data_withDates=data[(data['Date']>=datetime.date(2017,11,1))]
#test_data_xaxis = test_data_withDates['Date']
plt.tick_params(
    axis='x',          # changes apply to the x-axis
    which='both',      # both major and minor ticks are affected
    bottom=False,      # ticks along the bottom edge are off
    top=False,         # ticks along the top edge are off
    labelbottom=False) # labels along the bottom edge are off

plt.plot(test_data['DL_PRED_GRU'],'r', label='Prediction')
plt.plot(test_data['PV'],'g')
plt.ylabel('Energy (Normalized)',fontsize=12)
plt.xlabel('Date: November & December 2017',fontsize=12)
plt.legend()

In [ ]:
test_data['PV'].mean()

In [ ]:
x = test_data['PV']
y = test_data["DL_PRED_GRU"]

fig,ax = plt.subplots()
ax.scatter(y, x)
ax.plot([y.min(), y.max()], [y.min(), y.max()], 'r', lw=2)
ax.set_xlabel('Measured')
ax.set_ylabel('Predicted')
fig.show()